<a href="https://colab.research.google.com/github/AlonsoGG-dev/03MIAR---Algoritmos-de-Optimizacion---2024/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos_de_Optimizacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Alonso Garrido Gómez  <br>
Url: https://github.com/AlonsoGG-dev/03MIAR---Algoritmos-de-Optimizacion---2024/blob/main/TRABAJO_PRACTICO.ipynb<br>
Google Colab: https://drive.google.com/drive/u/5/folders/13hnJfpBFv09VPSeeJvSvI64R7KDdZDtx <br>
Problema:
*Organizar los horarios de partidos de una jornada de La Liga*

Descripción del problema:

Desde la liga de fútbol profesional se pretende organizar los horarios de los partidos de liga de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un algoritmo que realice la asignación de los partidos a los horarios de forma que maximice la audiencia.






                                        

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

**¿Como represento el espacio de soluciones?**

Cada solución es una asignación completa de partidos a franjas horarias.

Esto se puede representar como una lista de longitud N (donde N es el número de partidos) en el que cada posición indica el horario asignado a ese partido. Así, cada partido tiene H opciones (siendo H el número de franjas), y el espacio total de soluciones es de tamaño $H^N$


---


**¿Cual es la función objetivo?**

Se busca maximizar la audiencia total. La funcion objetivo para obtener la audiencia de cada partido se calcula:

$$
\text{Audiencia}(m, h) = \text{base}(m) \times \text{factor}(h) \times \text{penalty}(n_h)
$$

donde:

- base(m) es la audiencia base del partido m (según las categorías de los equipos)

- factor(h) es el impacto del horario h en la audiencia (por ejemplo, mayor audiencia en sábado a las 20h)

- penalty($n_h$) es el factor de penalización que depende del número $n_h$ de partidos que se juegan simultáneamente en ese mismo horario.

La función objetivo global es maximizar la suma de estas audiencias para todos los partidos.

---

**¿Como implemento las restricciones?**

- **Asignación única**: Cada partido debe asignarse a una única franja horaria, lo que se hace forzando que el vector de asignaciones tenga un único valor (entre 0 y H-1) para cada partido.

- **Restricción de viernes y lunes**: Se impone que al menos un partido se asigne a la franja de viernes y al menos uno al de lunes. En el algoritmo backtracking se puede definir una "flag" que verifique si ya se usó cada uno de estos horarios y, en la rama final, solo se aceptan soluciones que cumplan esta condición.

- **Penalización por concurrencia**: La penalización se aplica según el número de partidos asignados a la misma franja. Al asignar un partido, se actualiza de forma incremental el contador de partidos en ese horario y se recalcula la audiencia considerando el factor de penalización que corresponda a ese número de coincidencias.

- **Poda (Branch & Bound)**: Se utiliza una cota superior optimista para los partidos no asignados, de modo que si la solución parcial no puede superar la mejor solución hallada, se descarta esa rama y se evita seguir explorándola.


#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

El problema tiene complejidad exponencial. Cada partido (de un total de N) se puede asignar a cualquiera de las H franjas horarias, por lo que el espacio de soluciones es de tamaño: $H^N$

En este caso, si hay 10 partidos y 10 franjas, el número total de asignaciones posibles es $10^{10}$ (diez mil millones). Esto implica que, en el peor caso, la complejidad es
$$O(H^N)$$

#Diseño
- ¿Que técnica utilizo? ¿Por qué?

Tras analizar diferentes opciones, identifiqué tres enfoques con los que podría resolver este problema:

- **Backtracking con ramificación y poda**

**Ventaja**: Fácil de implementar desde cero; se asignan partidos recursivamente y, aplicando poda, se descartan ramas que no superen la mejor solución. Se obtiene la solución óptima

**Desventaja**: Si el número de partidos crece, puede volverse lento

- **Programación Lineal Entera (PLE)**

**Ventaja**: Con un solver (por ejemplo, CBC, GLPK) se obtiene la solucion óptima y certificada rápidamente.

**Desventaja**: Implica linealizar la penalización por coincidencia, lo que complica un poco el modelo, y depende de usar un solver externo (se puede implementar uno propio, pero es complejo).

- **Heurísticas de búsqueda local**

**Ventaja**: Permiten prototipar rápidamente generando soluciones iniciales y mejorándolas mediante recocido simulado o GRASP.

**Desventaja**: No garantizan optimalidad y la calidad depende de cómo se definan la vecindad y el criterio de parada.

---

**Conclusión:**

El mejor balance entre sencillez y eficiencia, considerando que en el curso se ha estudiado esta técnica es:

 *Backtracking con ramificación y poda*

 Es didáctico, claro de programar a mano y suficientemente eficiente para el tamaño del problema.

#Desarrollo del algoritmo

In [ ]:
import pandas as pd

# --------------------------------------------------
# 1. DATOS DEL PROBLEMA
# --------------------------------------------------

team_categories = {
    "Celta": "B",
    "Real Madrid": "A",
    "Valencia": "B",
    "R. Sociedad": "A",
    "Mallorca": "C",
    "Eibar": "C",
    "Athletic": "B",
    "Barcelona": "A",
    "Leganés": "C",
    "Osasuna": "C",
    "Villarreal": "B",
    "Granada": "C",
    "Alavés": "B",
    "Levante": "B",
    "Espanyol": "B",
    "Sevilla": "B",
    "Betis": "B",
    "Valladolid": "C",
    "Atlético": "B",
    "Getafe": "B"
}

# Partidos (base de audiencia en millones). Mejor escenario para 10 partidos
base_audience = [1.3, 1.3, 0.47, 1.3, 0.47, 0.75, 0.9, 0.9, 0.75, 0.9]
matches = [
    ("Celta", "Real Madrid"),
    ("Valencia", "R. Sociedad"),
    ("Mallorca", "Eibar"),
    ("Athletic", "Barcelona"),
    ("Leganés", "Osasuna"),
    ("Villarreal", "Granada"),
    ("Alavés", "Levante"),
    ("Espanyol", "Sevilla"),
    ("Betis", "Valladolid"),
    ("Atlético", "Getafe")
]

N = len(base_audience)

# Horarios disponibles y factor
day_factor = [0.4, 0.55, 0.7, 0.8, 1.0, 0.45, 0.75, 0.85, 1.0, 0.4]
timeslot_names = ["Vie20","Sab12","Sab16","Sab18","Sab20","Dom12","Dom16","Dom18","Dom20","Lun20"]
H = len(day_factor)

# Penalización por coincidencias:
penalty_factor = {
   1: 1.00,
   2: 0.75,
   3: 0.55,
   4: 0.40,
   5: 0.30,
   6: 0.25,
   7: 0.22,
   8: 0.20
}

"""
Devuelve el factor por concurrencia cuando hay n partidos en la misma franja
"""
def concurrency_factor(n):
    if n <= 1:
        return penalty_factor[1]
    elif n >= 8:
        return penalty_factor[8]
    else:
        return penalty_factor[n]


# --------------------------------------------------
# 2. ESTRUCTURAS PARA EL BACKTRACKING
# --------------------------------------------------

# Numero de partidos en cada horario
num_match = [0]*H
# Suma de las bases de audiencia en cada horario
sum_audience= [0.0]*H

best_audience = 0.0
best_assignment = None

# Para almacenar la asignación parcial de cada partido
current_assignment = [-1]*N  # -1 significa no asignado aún


# --------------------------------------------------
# 3. FUNCIÓN DE CÁLCULO INCREMENTAL
# --------------------------------------------------

"""
Añade el partido 'match_index' y devuelve la nueva audiencia total,
calculando el 'delta' que produce este cambio
"""
def add_match(match_index, slot, current_audience):
    old_n = num_match[slot]
    old_factor = concurrency_factor(old_n) if old_n>0 else 0.0
    old_sum_audience = sum_audience[slot]

    # Audiencia antes de añadir (la que aportaba este slot):
    Audience_old = old_factor * day_factor[slot] * old_sum_audience

    # Incrementar el slot
    new_n = old_n + 1
    new_factor = concurrency_factor(new_n)
    new_sum_audience = old_sum_audience + base_audience[match_index]

    # Audiencia nueva para este slot
    Audience_new = new_factor * day_factor[slot] * new_sum_audience

    # Delta (diferencia en la audiencia)
    delta = Audience_new - Audience_old

    # Actualizacion de estructuras
    num_match[slot] = new_n
    sum_audience[slot] = new_sum_audience

    return current_audience + delta


"""
Operación inversa para backtracking: quita el partido del slot
y devuelve la audiencia restaurada calculando el delta inverso
"""
def remove_match(match_index, slot, previous_audience):
    new_n = num_match[slot]
    new_sum_audience = sum_audience[slot]

    # Audiencia actual en el slot (después de haber añadido):
    Audience_current = concurrency_factor(new_n) * day_factor[slot] * new_sum_audience

    # Revertir el slot
    old_n = new_n - 1
    old_sum_audience = new_sum_audience - base_audience[match_index]

    # Audiencia anterior del slot
    Audience_old = concurrency_factor(old_n) * day_factor[slot] * old_sum_audience

    delta = Audience_current - Audience_old

    # Actualización de estructuras
    num_match[slot] = old_n
    sum_audience[slot] = old_sum_audience

    return previous_audience - delta


# --------------------------------------------------
# 4. FUNCIÓN DE COTA (BOUND) PARA PODA
# --------------------------------------------------

"""
Calcula una cota superior asumiendo que los partidos no asignados
podrían lograr la máxima audiencia individual sin penalización
"""
def upper_bound(match_idx, current_audience):

    # Es un bound muy optimista, pero suficiente para podar algo.
    remaining_audience = sum(base_audience[match_idx:])
    return current_audience + remaining_audience


# --------------------------------------------------
# 5. BACKTRACKING RECURSIVO
# --------------------------------------------------
def backtrack(match_idx, current_audience, used_friday, used_monday):
    global best_audience, best_assignment

    # Si ya se asignaron todos los partidos, comprobar la restricción (viernes y lunes)
    if match_idx == N:
        if used_friday and used_monday:
            if current_audience > best_audience:
                best_audience = current_audience
                best_assignment = current_assignment[:]
        return

    # Chequear cota para poda
    ub = upper_bound(match_idx, current_audience)
    if ub < best_audience:
        # No vale la pena seguir por aquí
        return

    # Intentar asignar el partido 'match_idx' a cada horario posible
    for slot in range(H):
        # Calcular la nueva audiencia si se pone match_idx en 'slot'
        new_audience = add_match(match_idx, slot, current_audience)

        # Actualizar la asignación
        current_assignment[match_idx] = slot

        # Actualizar banderas de viernes y lunes
        new_used_friday = used_friday or (slot == 0)
        new_used_monday = used_monday or (slot == 9)

        # Llamada recursiva
        backtrack(match_idx + 1, new_audience, new_used_friday, new_used_monday)

        # Deshacer la asignación (backtrack)
        restored_audience = remove_match(match_idx, slot, new_audience)
        current_assignment[match_idx] = -1


# --------------------------------------------------
# 6. EJECUCIÓN DEL BACKTRACKING
# --------------------------------------------------
if __name__ == "__main__":
    # Llamar con match_idx=0, audience=0, usedFriday=False, usedMonday=False
    backtrack(0, 0.0, False, False)

    if best_assignment is not None:
        # Contar cuántos partidos hay asignados a cada horario para la corrección de coincidencias
        slot_counts = [0] * H
        for slot in best_assignment:
            slot_counts[slot] += 1

        data = []
        for i, slot in enumerate(best_assignment):
            # Construir la cadena de categorías para el partido i
            cat_local = team_categories[matches[i][0]]
            cat_visitante = team_categories[matches[i][1]]
            categorias = f"{cat_local} vs {cat_visitante}"

            ponderacion = day_factor[slot]
            audiencia_base = base_audience[i]
            audiencia_ponderada = audiencia_base * ponderacion
            fc = concurrency_factor(slot_counts[slot])
            correction = audiencia_ponderada * fc

            data.append({
                "Partido": f"{matches[i][0]} vs {matches[i][1]}",
                "Categorías": categorias,
                "Horario": timeslot_names[slot],
                "Audiencia (Mill.)": audiencia_base,
                "Ponderación": ponderacion,
                "Audiencia*Ponderación": round(audiencia_ponderada, 2),
                "Corrección Coincidencia": round(correction, 2)
            })
        df = pd.DataFrame(data)
        print(f"Audiencia total conseguida: {best_audience:.2f} millones de espectadores")
        display(df)
    else:
        print("No se encontró ninguna asignación que cumpla con el partido en viernes y lunes.")

Audiencia total conseguida: 6.86 millones de espectadores


,Partido,Categorías,Horario,Audiencia (Mill.),Ponderación,Audiencia*Ponderación,Corrección Coincidencia
0,Celta vs Real Madrid,B vs A,Dom20,1.30,1.00,1.30,1.30
1,Valencia vs R. Sociedad,B vs A,Dom18,1.30,0.85,1.10,1.10
2,Mallorca vs Eibar,C vs C,Lun20,0.47,0.40,0.19,0.19
3,Athletic vs Barcelona,B vs A,Sab20,1.30,1.00,1.30,1.30
4,Leganés vs Osasuna,C vs C,Vie20,0.47,0.40,0.19,0.19
5,Villarreal vs Granada,B vs C,Sab12,0.75,0.55,0.41,0.41
6,Alavés vs Levante,B vs B,Sab16,0.90,0.70,0.63,0.63
7,Espanyol vs Sevilla,B vs B,Sab18,0.90,0.80,0.72,0.72
8,Betis vs Valladolid,B vs C,Dom12,0.75,0.45,0.34,0.34
9,Atlético vs Getafe,B vs B,Dom16,0.90,0.75,0.68,0.68
